In [ ]:
import os
from os.path import exists, join, basename, splitext
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')
#os.chdir("/content/gdrive/My Drive/Drone_Recog/")

Mounted at /content/gdrive


# Installing OpenPose

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


install new CMake becaue of CUDA10
clone openpose
install system dependencies
Selecting previously unselected package libgflags2.2.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Select

In [ ]:
#@title Install Body 25B Model
!mkdir /content/openpose/models/pose/body_25b
!wget http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/1_25BBkg/body_25b/pose_iter_XXXXXX.caffemodel -O /content/openpose/models/pose/body_25b/pose_iter_XXXXXX.caffemodel
!wget https://raw.githubusercontent.com/CMU-Perceptual-Computing-Lab/openpose_train/master/experimental_models/1_25BBkg/body_25b/pose_deploy.prototxt -O /content/openpose/models/pose/body_25b/pose_deploy.prototxt

--2022-04-12 16:19:27--  http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/1_25BBkg/body_25b/pose_iter_XXXXXX.caffemodel
Resolving posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)... 128.2.220.57
Connecting to posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)|128.2.220.57|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106928170 (102M) [text/plain]
Saving to: ‘/content/openpose/models/pose/body_25b/pose_iter_XXXXXX.caffemodel’

/content/openpose/m 100%[===================>] 101.97M  10.1MB/s    in 13s     

2022-04-12 16:19:41 (7.67 MB/s) - ‘/content/openpose/models/pose/body_25b/pose_iter_XXXXXX.caffemodel’ saved [106928170/106928170]

--2022-04-12 16:19:41--  https://raw.githubusercontent.com/CMU-Perceptual-Computing-Lab/openpose_train/master/experimental_models/1_25BBkg/body_25b/pose_deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connec

# OpenPose on Videos

## Extract open pose data

Video Files need to be moved to contents before hand and having it in My Drive makes extracting open pose results harder

In [2]:
!ls Actions

boxing		hitting_stick  kicking	    running_side  walking_side
clapping	jogging_f_b    OpenPose     stabbing	  waving_hands
hitting_bottle	jogging_side   running_f_b  walking_f_b


In [24]:
# in case there is a mistake
!rm -rf /content/Actions/hitting_bottle/.ipynb_checkpoints


In [3]:
# in content
!mkdir /content/Actions/OpenPose


mkdir: cannot create directory ‘/content/Actions/OpenPose’: File exists


In [28]:
# OpenPose on a video under different directories
import os
from os.path import exists, join, basename, splitext

h, w = 480, 720

vid_path = "/content/Actions"
class_names = os.listdir(vid_path)
num_class = len(class_names)

openpose_out = '/content/Actions/OpenPose'
openpose_json = '/content/OP_Results/json'

# save json files and video into another directory
for idx, cname in enumerate(class_names):
  if idx != 4:
    class_path = os.path.join(vid_path, cname)
    video_names = sorted(os.listdir(class_path))
    for vname in video_names:
        video_path = os.path.join(class_path, vname)
        out_vidpath = os.path.join(openpose_out, cname, vname)
        
        if not os.path.exists(os.path.join(openpose_out, cname)):
            os.makedirs(os.path.join(openpose_out, cname))

        if not os.path.exists(out_vidpath):
          !cd /content/openpose && ./build/examples/openpose/openpose.bin \
          --net_resolution -1x{h} --scale_number 4 --scale_gap 0.25 \
          --video {video_path} \
          --number_people_max 1 \
          --display 0 \
          --write_video {out_vidpath}

#if not os.path.exists(json_vidpath):
    #os.makedirs(json_vidpath)
#json_vidpath = os.path.join(openpose_json, cname, os.path.splitext(vname)[0])
#--write_json {json_vidpath} \

20
19
20
20
10
20
20
20
20
10


## Combining JSON Files

Move the OP_results folder with all the json and vOP videos back to yout GDrive, in the same directory as your datasets

In [ ]:
from genericpath import isfile
import os
import shutil
import glob
import json

#!ls OP_Results
json_path = "OP_Results/json"
combined_json_path = "OP_Results/combined_json"

class_names = os.listdir(json_path)
num_class = len(class_names)

for cname in class_names:
    class_path = os.path.join(json_path, cname)
    combined_path = os.path.join(combined_json_path, cname)
    video_names = sorted(os.listdir(class_path))
    
    for vname in video_names:
        vid_path = os.path.join(class_path, vname)
        combined_vidpath = os.path.join(combined_path, vname)
        if not os.path.exists(combined_vidpath):
            os.makedirs(combined_vidpath)
            print('Creating new directory in combined JSON: vname')
        print(f'{vname} frame count: {len(os.listdir(vid_path))}')

        data = []
        for name in sorted(glob.glob(vid_path + '/*_keypoints.json')):
            try:
                f = open(name, )
                d = json.load(f)
                data.append([os.path.basename(name), d['people']])
            except ValueError:
                print(f'Decoding JSON has failed: {name}')
        
        with open(f'{combined_vidpath}/{os.path.splitext(vname)[0]}.json', 'w') as outfile:
            json.dump(data, outfile)

In [ ]:
from json import load
import os
import json

# Try loading json files
combined_json_path = "OP_Results/combined_json"
class_names = os.listdir(combined_json_path)

example_json = f'{combined_json_path}/waving_hands/S1_wavingHands_HD/S1_wavingHands_HD.json'

f = open(example_json)
data = json.load(f)

print(data)

f.close()

[['S1_wavingHands_HD_000000000000_keypoints.json', [{'person_id': [-1], 'pose_keypoints_2d': [957.934, 707.321, 0.917678, 966.933, 701.292, 0.93301, 951.936, 699.84, 0.91417, 977.432, 705.823, 0.851461, 941.439, 704.324, 0.843016, 992.45, 746.338, 0.898618, 926.438, 747.827, 0.88801, 1011.95, 794.382, 0.8883, 914.414, 798.845, 0.895132, 983.474, 821.402, 0.81611, 933.946, 830.366, 0.827792, 980.475, 852.89, 0.788844, 941.41, 855.898, 0.838917, 978.978, 927.907, 0.849393, 942.925, 927.906, 0.889342, 981.974, 990.98, 0.84398, 944.436, 990.934, 0.863666, 957.967, 729.86, 0.863654, 959.433, 675.807, 0.880157, 990.985, 1014.98, 0.848302, 1001.45, 1010.47, 0.852559, 977.456, 995.473, 0.802471, 942.923, 1013.46, 0.826622, 930.94, 1008.95, 0.796562, 950.435, 992.489, 0.738431], 'face_keypoints_2d': [], 'hand_left_keypoints_2d': [], 'hand_right_keypoints_2d': [], 'pose_keypoints_3d': [], 'face_keypoints_3d': [], 'hand_left_keypoints_3d': [], 'hand_right_keypoints_3d': []}]], ['S1_wavingHands_HD

# OpenPose Live Images

*Go here*
https://colab.research.google.com/github/changsin/DL/blob/main/notebooks/openpose_sign_language.ipynb#scrollTo=PMDlY-h__z_g